# --- Day 15: Warehouse Woes ---


In [ ]:
# --- Part One ---

# from icecream import ic
# import re

filename = "input.txt"

#filename = "test.txt" # decomment to use test
#filename = "test_mini.txt" # decomment to use test

def parse_file(file_path):
    grid = { "size": (0,0), "walls": [], "boxes" : [], "robot" : None}
    movements = [] # 

    with open(file_path, 'r') as file:
        #
        content = file.read()
    blocks = [block.strip() for block in content.split("\n\n") if block.strip()]

    # get grid  

    gridmap = [list(c) for c in blocks[0].splitlines()]
    grid["size"] = (len(gridmap[0]), len(gridmap))

    for y, row in enumerate(gridmap):
        for x, obj in enumerate(row):
            if obj == "@":
                grid["robot"] = (x,y)
            elif obj == "#":
                grid["walls"].append((x,y))
            elif obj == "O":
                grid["boxes"].append((x,y))
            else:
                pass
    # get movements
    for line in blocks[1].splitlines():
        movements.extend(list(line.strip()))
            
    return grid, movements

class FieldMap:

    def __init__(self, grid):
        self.walls = grid["walls"]
        self.boxes = grid["boxes"]
        self.robot_pos = grid["robot"]
        self.size = grid["size"]

    def __str__(self):
        retstr = ""
        for y in range(self.size[1]):
            for x in range(self.size[0]):
                if (x,y) in self.walls:
                    retstr += "#"
                elif (x,y) in self.boxes:
                    retstr += "O"
                elif (x,y) == self.robot_pos:
                    retstr += "@"
                else:
                    retstr += "."
            retstr += "\n"
        return retstr

    def _new_pos(self, pos, dir):
        if dir == "<":
            return (pos[0]-1, pos[1])
        if dir == ">":
            return (pos[0]+1, pos[1])
        if dir == "^":
            return (pos[0], pos[1]-1)
        if dir == "v":
            return (pos[0], pos[1]+1)
        return None
        
    def move_object(self, position, direction) -> bool:
        if position[0] < 0 or position[0] >= self.size[0]:
            return False
        if position[1] < 0 or position[1] >= self.size[1]:
            return False
        if position in self.walls:
            return False
        if position in self.boxes:
            new_pos = self._new_pos(position, direction)
            if self.move_object(new_pos , direction):
                self.boxes.remove(position)
                self.boxes.append(new_pos)
                return True
            return False        
        if position == self.robot_pos:        
            new_pos = self._new_pos(position, direction)
            if self.move_object(new_pos , direction):
                self.robot_pos = new_pos
                return True
            return False                
        return True        

# ---- main ----

grid, movements = parse_file(filename)

fieldmap = FieldMap(grid)

#print(fieldmap)

for mv in movements:
    fieldmap.move_object(fieldmap.robot_pos, mv)
#    print(f"Move {mv}:")
    
#print(fieldmap)    

ret1 = sum([100 * v[1] + v[0] for v in fieldmap.boxes])

print (f"The solution of for part 1 is: {ret1}")


The solution of for part 1 is: 1509074


In [58]:
# --- Part two ---
# Uses code of Part One


# from icecream import ic
# import re

filename = "input.txt"

#filename = "test.txt" # decomment to use test
#filename = "test2_mini.txt" # decomment to use test

def parse_file(file_path):
    grid = { "size": (0,0), "walls": [], "boxes" : {}, "robot" : None}
    movements = [] # 

    with open(file_path, 'r') as file:
        #
        content = file.read()
    blocks = [block.strip() for block in content.split("\n\n") if block.strip()]

    # get grid  

    gridmap = [list(c) for c in blocks[0].splitlines()]
    grid["size"] = (2 * len(gridmap[0]), len(gridmap))

    for y, row in enumerate(gridmap):
        for x, obj in enumerate(row):
            if obj == "@":
                grid["robot"] = (x*2,y)
            elif obj == "#":
                grid["walls"].append((x*2,y))
                grid["walls"].append((x*2+1,y))
            elif obj == "O":
                grid["boxes"][(x*2,y)] = "["
                grid["boxes"][(x*2+1,y)] = "]"
            else:
                pass
    # get movements
    for line in blocks[1].splitlines():
        movements.extend(list(line.strip()))
            
    return grid, movements

class FieldMap:

    def __init__(self, grid):
        self.walls = grid["walls"]
        self.boxes = grid["boxes"]
        self.robot_pos = grid["robot"]
        self.size = grid["size"]

    def __str__(self):
        retstr = ""
        for y in range(self.size[1]):
            for x in range(self.size[0]):
                if (x,y) in self.walls:
                    retstr += "#"
                elif (x,y) in self.boxes:
                    retstr += self.boxes[(x,y)]
                elif (x,y) == self.robot_pos:
                    retstr += "@"
                else:
                    retstr += "."
            retstr += "\n"
        return retstr

    def _new_pos(self, pos, dir):
        if dir == "<":
            return (pos[0]-1, pos[1])
        if dir == ">":
            return (pos[0]+1, pos[1])
        if dir == "^":
            return (pos[0], pos[1]-1)
        if dir == "v":
            return (pos[0], pos[1]+1)
        return None

    def _can_move(self, position, direction):
        if position[0] < 0 or position[0] >= self.size[0]:
            return False
        if position[1] < 0 or position[1] >= self.size[1]:
            return False
        if position in self.walls:
            return False
        if position in self.boxes:
            if self.boxes[position] == "[":
                pos_l = position
                pos_r = self._new_pos(position, ">")
            else:
                pos_r = position
                pos_l = self._new_pos(position, "<")
            if direction == "<":
                new_pos = self._new_pos(pos_l, direction)
                if self._can_move(new_pos, direction):
                    return True
            elif direction == ">":
                new_pos = self._new_pos(pos_r, direction)
                if self._can_move(new_pos, direction):
                    return True
            else: # direction "^" or "v"            
                new_pos_l = self._new_pos(pos_l, direction)
                new_pos_r = self._new_pos(pos_r, direction)
                if  self._can_move(new_pos_l , direction) and self._can_move(new_pos_r , direction):
                    return True
            return False        
        if position == self.robot_pos:        
            new_pos = self._new_pos(position, direction)
            if self._can_move(new_pos , direction):
                return True
            return False                
        return True        
        
    def move_object(self, position, direction) -> bool:
        if position in self.boxes:
            if self._can_move(position, direction):
                if self.boxes[position] == "[":
                    pos_l = position
                    pos_r = self._new_pos(position, ">")
                else:
                    pos_r = position
                    pos_l = self._new_pos(position, "<")
                new_pos_l = self._new_pos(pos_l, direction)
                new_pos_r = self._new_pos(pos_r, direction)
                if direction == "<":
                    self.move_object(new_pos_l, direction)
                elif direction == ">":
                    self.move_object(new_pos_r, direction)
                else:
                    self.move_object(new_pos_l, direction)
                    self.move_object(new_pos_r, direction)
                del self.boxes[pos_l]
                del self.boxes[pos_r]
                self.boxes[new_pos_l] = "["
                self.boxes[new_pos_r] = "]"
                return True
            return False
        if position == self.robot_pos:        
            if self._can_move(position, direction):
                new_pos = self._new_pos(position, direction)
                if self.move_object(new_pos, direction):
                    self.robot_pos = new_pos
                return True
            return False                
        return True        

# ---- main ----

grid, movements = parse_file(filename)

fieldmap = FieldMap(grid)

# print(fieldmap)

for mv in movements:
    fieldmap.move_object(fieldmap.robot_pos, mv)
    #print(f"Move {mv}:")

# print(fieldmap)    

ret2 = sum([100 * k[1] + k[0] for k, v in fieldmap.boxes.items() if v == "["])

print (f"The solution of for part 2 is: {ret2}")


The solution of for part 2 is: 1521453
